In [1]:
from tensorflow.keras.datasets.fashion_mnist import load_data
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
class Mylearning:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = self.data_reshape(x_train)
        self.y_train = y_train
        self.x_test = self.data_reshape(x_test)
        self.y_test = y_test
    
    def get_prm(self):
        return self.x_train, self.y_train, self.x_test, self.y_test
    
    def data_reshape(self, x):
        x = x.reshape(len(x), 28 * 28)
        x = x / 255.0
        return x

In [3]:
(x_train, y_train), (x_test, y_test) = load_data()
model = Mylearning(x_train, y_train, x_test, y_test)
x, y, xt, yt = model.get_prm()